# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他數值型欄位一起預估生存機率

In [53]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [54]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [55]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=10).mean())

0.7777343661332426


In [56]:
# 'Cabin'標籤編碼 + 'Cabin'計數編碼 + 邏輯斯迴歸
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
df = pd.merge(df, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Count'] = df['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=10).mean())

0.7900692316422654


C:\Users\K258\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=10).mean())

0.7822290318919533


In [58]:
# 'Cabin'標籤編碼 + 'Cabin'計數編碼  + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
df_temp['Cabin'] = df['Cabin_Count']
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=10).mean())
df_temp

0.7934655544206105


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,1014,3,6
1,286,0,816,2,0,0
2,523,0,914,1014,3,6
3,422,0,65,2,3,4
4,22,1,649,1014,3,6
5,818,1,373,1014,2,6
6,767,1,109,2,3,2
7,914,1,541,1014,3,6
8,605,0,477,1014,3,6
9,847,0,174,1014,0,6


In [81]:
# 'Cabin'標籤編碼 +均值編碼 + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
mean_df = data.groupby(['Cabin'])['Survived'].mean().reset_index()
mean_df.columns = ['Cabin', f"{'Cabin'}_mean"]
data = pd.merge(data, mean_df, on='Cabin', how='left')
data = data.drop(['Survived','Cabin'] , axis=1)
df_temp = pd.DataFrame()
for c in data.columns[:4]:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
train_X = pd.concat([train_X, data['Cabin_mean']], axis=1)
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=10).mean())
train_X.head()

0.835001418681194


,Name,Sex,Ticket,Embarked,Cabin_mean
0,155,1,720,3,0.299854
1,286,0,816,0,1.000000
2,523,0,914,3,0.299854
3,422,0,65,3,0.500000
4,22,1,649,3,0.299854


In [82]:
print("標籤編碼:0.7777343661332426")
print("標籤編碼+計數編碼:0.7900692316422654")
print("特徵雜湊:0.7822290318919533")
print("特徵雜湊+標籤編碼+計數編碼:0.7934655544206105")
print("標籤編碼 + 均值編碼:0.835001418681194")
print("均值編碼:效果最好 也許可以推斷cabin類型與存活率有很大關聯")

標籤編碼:0.7777343661332426
標籤編碼+計數編碼:0.7900692316422654
特徵雜湊:0.7822290318919533
特徵雜湊+標籤編碼+計數編碼:0.7934655544206105
標籤編碼 + 均值編碼:0.835001418681194
均值編碼:效果最好 也許可以推斷cabin類型與存活率有很大關聯
